# DS-SF-34 | 11 | Logistic Regression | Assignment | Starter Code

In [99]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 30)

from sklearn import linear_model, model_selection
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Probability, Odds, and Odds Ratios

**Probability:** The number of ways that an event can occur divided by the total number of possible outcomes.

The probability of drawing a red card from a standard deck of cards is 26/52 (50 percent).
The probability of drawing a club from that deck is 13/52 (25 percent).

> ### Question 1.  What's the probability of getting heads in a fair coin flip?

In [3]:
# TODO
p=1./2
print p

0.5


The odds for an event is the ratio of the number of ways the event can occur compared to the number of ways it does not occur.

For example, using the same events as above, the odds for:
- Drawing a red card from a standard deck of cards is 1:1; and
- Drawing a club from that deck is 1:3.

> ### Question 2.  What's the odds of a fair coin flip?

In [ ]:
# TODO
1:1

> ### Question 3.  Suppose that 18 out of 20 patients in an experiment lost weight while using diet A, while 16 out of 20  lost weight using diet B.  What's the probability of weight loss with diet A?  What's the odds?

In [ ]:
# TODO
probability = 18/20
odds of weight loss with diet A = 18:2  9:1   

> ### Question 4.  What's the probablity of weight loss with diet B?  What are the odds?

In [ ]:
# TODO
probability = 16/20
odds of weight loss with diet B = 16:4   4:1

> ### Question 5.  What's the odds ratio?

In [5]:
# TODO
# odds ratio of diet A vs diet B
odds = 9/4.
print odds

2.25


## Bank Marketing

In [143]:
df = pd.read_csv(os.path.join('..', 'datasets', 'dataset-11-bank-marketing.csv'))
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,c
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


> The dataset is related to the direct marketing campaigns (by phone) of a Portuguese banking institution.  The classification goal is to predict if the client will subscribe a term deposit (variable y).

Attributes Information:

- Input variables:
  - [Bank client data]
    - `age` (numeric)
    - `job`: type of job (categorical)
    - `marital`: marital status (categorical)
      - Note: `divorced` means divorced or widowed)
    - `education` (categorical)
    - `default`: has credit in default? (categorical)
    - `balance`: bank account balance (\$)
    - `housing`: has housing loan? (categorical)
    - `loan`: has personal loan? (categorical)
  - [Data related with the last contact of the current campaign]
    - `contact`: contact communication type (categorical)
    - `month`: last contact month of year (categorical)
    - `day_of_week`: last contact day of the week (categorical)
    - `duration`: last contact duration, in seconds (numeric)
      - Important note: this attribute highly affects the output target (e.g., if `duration = 0` then `y = 'no'`). Yet, the duration is not known before a call is performed.  Also, after the end of the call y is obviously known.  Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
  - [Other attributes]
    - `campaign`: number of contacts performed during this campaign and for this client (numeric)
    - `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric)
      - 999 means client was not previously contacted
    - `previous`: number of contacts performed before this campaign and for this client (numeric)
    - `poutcome`: outcome of the previous marketing campaign (categorical)

- Output variable (desired target):
  - `y`: has the client subscribed a term deposit? (binary)

In [128]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
c            object
dtype: object

> Our goal is to develop a model the best predicts the outcome `y`, the success of the marketing campaign.

> ## Question 6.  Remove the categorical variables with the most number of distinct values

In [67]:
# TODO  
   # 12   -- this one has the most number of distinct values
#len(df.job.value_counts(sort=False))
   # 3  married  2797 divorced   528 single  1196
# df.marital.value_counts(sort=False)

# 4   secondary 2306   primary 678  tertiary   1350   UNKNOWN 187
df.education.value_counts(sort=False)

# 2  yes  76 no 4445  -- turn this into default_no    
#df.default.value_counts(sort=False)

# 2  yes value 2559   no values 1962  -- leave this as housing
# df.housing.value_counts(sort=False)

# 2  yes 691 no  3830  -- turn this into loan_no
# df.loan.value_counts(sort=False)

   # 3   cellular   2896 UNKNOWN  1324 telephone  301
# df.contact.value_counts(sort=False)

# 4   success   129  UNKNOWN    3705 failure   490 other 197
#df.poutcome.value_counts(sort=False)

# 2   yes 521 no 4000  -- turn this into subscribed_no
#df.c.value_counts(sort=False)


secondary    2306
primary       678
tertiary     1350
unknown       187
Name: education, dtype: int64

In [137]:
df.drop(['job'], axis = 1, inplace = True)


> ## Question 7.  Recode all `yes`/`no` categorical variables with `0` as the most frequent value (then also append `"_no"` to the variable name), and `1` for the other (then leave the name unchanged)

In [144]:
# TODO
  # default
df['default'] = (df.default=='yes')*1
   # housing
df['housing_no'] = (df.housing=='no')*1
   # loan
df['loan'] = (df.loan=='yes')*1
df['c'] = (df.c=='yes')*1.
    

In [152]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,c,housing_no,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown
0,30,unemployed,married,primary,0,1787,no,0,cellular,19,oct,1,-1,0,unknown,0.0,1,0,1,0,1,0,0,0,1,0,0
1,33,services,married,secondary,0,4789,yes,1,cellular,11,may,1,339,4,failure,0.0,0,0,1,0,0,1,0,0,1,0,0
2,35,management,single,tertiary,0,1350,yes,0,cellular,16,apr,1,330,1,failure,0.0,0,0,0,1,0,0,1,0,1,0,0
3,30,management,married,tertiary,0,1476,yes,1,unknown,3,jun,4,-1,0,unknown,0.0,0,0,1,0,0,0,1,0,0,0,1
4,59,blue-collar,married,secondary,0,0,yes,0,unknown,5,may,1,-1,0,unknown,0.0,0,0,1,0,0,1,0,0,0,0,1
5,35,management,single,tertiary,0,747,no,0,cellular,23,feb,2,176,3,failure,0.0,1,0,0,1,0,0,1,0,1,0,0
6,36,self-employed,married,tertiary,0,307,yes,0,cellular,14,may,1,330,2,other,0.0,0,0,1,0,0,0,1,0,1,0,0
7,39,technician,married,secondary,0,147,yes,0,cellular,6,may,2,-1,0,unknown,0.0,0,0,1,0,0,1,0,0,1,0,0
8,41,entrepreneur,married,tertiary,0,221,yes,0,unknown,14,may,2,-1,0,unknown,0.0,0,0,1,0,0,0,1,0,0,0,1
9,43,services,married,primary,0,-88,yes,1,cellular,17,apr,1,147,2,failure,0.0,0,0,1,0,1,0,0,0,1,0,0


> ## Question 8.  Create binary/dummy variables for the other categorical variables

In [145]:
# TODO
marital_df = pd.get_dummies(df.marital, prefix='marital')
education_df = pd.get_dummies(df.education, prefix='education')
contact_df = pd.get_dummies(df.contact, prefix='contact')
df = df.join([marital_df, education_df, contact_df])


> ## Question 9.  What should be your baseline for these categorical variables (namely, which binary variables should you not include in your model)?

In [ ]:
# TODO  the baseline values should be marital = married, education = secondary, and contact = cellular, because 
# these are the most common values.

Answer: TODO

> ## Question 10.  What input variable in the dataset seems to predict the outcome quite well.  Why?

In [92]:
# TODO   duration seems to correlate with loan, because only when there is a loan can there be a non-null duration value.
df.corr().c.sort_values(ascending = False).head()

c                   1.000000
duration            0.401118
contact_cellular    0.118761
previous            0.116714
pdays               0.104087
Name: c, dtype: float64

Answer: TODO

In [146]:
df.drop('duration', axis = 1, inplace = True)

> ## Question 11.  Split the dataset into a training set (60%) and a testing set (the rest)

In [147]:
# TODO
train_df = df.sample(frac = .6, random_state = 0)
test_df = df.drop(train_df.index)

> ## Question 12.  Run a logistic regression with `age`, `marital` (the dummies), `default`, `balance`, `housing`, `loan`, `campaign`, `pdays`, `previous`?

In [135]:
# TODO
train_df.columns

Index([u'age', u'marital', u'education', u'balance', u'housing', u'contact',
       u'day', u'month', u'campaign', u'pdays', u'previous', u'poutcome', u'c',
       u'housing_no', u'marital_divorced', u'marital_married',
       u'marital_single', u'education_primary', u'education_secondary',
       u'education_tertiary', u'education_unknown', u'contact_cellular',
       u'contact_telephone', u'contact_unknown'],
      dtype='object')

In [148]:
names_X = ['age', 'marital_single', 'marital_divorced',
    'default', 'balance', 'housing_no',
    'loan', 'campaign', 'pdays', 'previous']

train_X = train_df[names_X]
train_c = train_df.c

model = linear_model.LogisticRegression().\
    fit(train_X, train_c)

print model.intercept_
print model.coef_
print cross_val_score(model,train_X, train_c, cv=5).mean()

test_X = test_df[names_X]
test_c = test_df.c

model_test = linear_model.LogisticRegression().\
    fit(test_X, test_c)

[-2.86298813]
[[  9.69884899e-03   4.59957633e-01   4.42239480e-01  -1.98476507e-02
    2.37763032e-05   6.49536640e-01  -7.26062474e-01  -6.16301237e-02
    1.64221565e-03   7.57725524e-02]]
0.882418299321


> ## Question 13.  What is your training error?  What is your generalization error?  Does it make sense?

In [150]:
# TODO
  # training error  0.116844821231
print 1 - model.score(train_X, train_c)
  # test error   0.117950608183
#print 1 - model_test.score(train_X, train_c)

0.116844821231


Answer: TODO

> ## Question 14.  Interpret your coefficients. (At least `marital_single`, `campaign`, and `default`).  Does your interpretation  make sense?

In [151]:
# TODO
zip(names_X, np.exp(model.coef_[0]) - 1)

[('age', 0.009746035248270335),
 ('marital_single', 0.58400687401399076),
 ('marital_divorced', 0.556188372406762),
 ('default', -0.019651982777419863),
 ('balance', 2.3776585814871964e-05),
 ('housing_no', 0.91465345031106504),
 ('loan', -0.51618974000724949),
 ('campaign', -0.059769408460234819),
 ('pdays', 0.0016435648238748435),
 ('previous', 0.078717194595496531)]

Answer: The odds that single people are targeted successfully by this campaign is e^.58 = 1.78 more likely than for married people. (Married = 0, single = 1)

The odds that a campaign is successful is a little less than 50/50 after contacting a person through an additional campaign. e^(-.06) = .94

The odds that a campaign is successful is about 50/50 after a person defaults on a loan. e^(-.02) = .98

> ## Question 15.  What is your prediction for a 30 years old single female, a homeowner with a \$1,000 balance in the bank, without a loan, who has never been contacted before, and who has never defaulted

In [117]:
# TODO

# names_X = ['age', 'marital_single', 'marital_divorced',
#     'default_no', 'balance', 'housing',
#     'loan_no', 'campaign', 'pdays', 'previous']

predict_X = [[30,1,0,1,1000,1,1,0,999,0]]
print model.predict(predict_X)
print model.predict_proba(predict_X)

# she has a 39% chance of subscribing for a term deposit.

[ 0.]
[[ 0.60619146  0.39380854]]


Answer: TODO